In [13]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
import time
from datetime import datetime
# Import the API key.
from config import weather_api_key

In [14]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
# zip packs each data point object that has the same index in each array into a tuple (0, 1)
lat_lngs = zip(lats, lngs)
lat_lngs

In [15]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [16]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

600

In [18]:
# Starting URL for Weather Map API Call.
# units = imperial will give us F not celsius
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [19]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
## initialized to begin at 1 since we want the iterations to start at 1 later
record_count = 1
set_count = 1

# Loop through all the cities in the list.
# enumerate will allow us to pull the city name value based off index position
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        # the one minute sleep after 50 cities will prevent us from exceeding the 60 call per minute free tier limit
        time.sleep(60)

    # Create endpoint URL with each city.
    ## we are removing blank spaces in city names and replacing with url style concats
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | taltal
Processing Record 2 of Set 1 | kapaa
Processing Record 3 of Set 1 | srednekolymsk
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | bluff
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | kibre mengist
Processing Record 9 of Set 1 | nichinan
Processing Record 10 of Set 1 | tiverton
Processing Record 11 of Set 1 | longyearbyen
Processing Record 12 of Set 1 | barrow
Processing Record 13 of Set 1 | makakilo city
Processing Record 14 of Set 1 | promyshlennyy
Processing Record 15 of Set 1 | aklavik
Processing Record 16 of Set 1 | goderich
Processing Record 17 of Set 1 | luau
Processing Record 18 of Set 1 | new richmond
Processing Record 19 of Set 1 | punta arenas
Processing Record 20 of Set 1 | salalah
Processing Record 21 of Set 1 | corn island
Processing Record 22 of Set 1 | husavik
Processing Record 23 of Set 1 | s

In [20]:
len(city_data)

547

In [21]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Taltal,-25.4000,-70.4833,66.67,53,8,4.90,CL,2022-09-03 18:11:20
1,Kapaa,22.0752,-159.3190,81.10,79,100,14.97,US,2022-09-03 18:11:20
2,Srednekolymsk,67.4500,153.6833,33.33,84,100,7.02,RU,2022-09-03 18:11:21
3,Albany,42.6001,-73.9662,82.38,54,96,3.00,US,2022-09-03 18:11:21
4,Qaanaaq,77.4840,-69.3632,38.53,98,100,10.00,GL,2022-09-03 18:11:22
5,Bluff,-46.6000,168.3333,41.81,77,87,38.12,NZ,2022-09-03 18:11:22
6,Ushuaia,-54.8000,-68.3000,37.06,93,100,6.91,AR,2022-09-03 18:11:22
7,Kibre Mengist,5.8833,38.9833,62.37,75,99,3.24,ET,2022-09-03 18:11:23
8,Nichinan,31.6000,131.3667,77.13,86,100,5.23,JP,2022-09-03 18:11:23
9,Tiverton,50.9000,-3.4833,65.05,92,75,9.22,GB,2022-09-03 18:11:23


In [23]:
cols = ['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']

city_data_df = city_data_df[cols]

city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Taltal,CL,2022-09-03 18:11:20,-25.4000,-70.4833,66.67,53,8,4.90
1,Kapaa,US,2022-09-03 18:11:20,22.0752,-159.3190,81.10,79,100,14.97
2,Srednekolymsk,RU,2022-09-03 18:11:21,67.4500,153.6833,33.33,84,100,7.02
3,Albany,US,2022-09-03 18:11:21,42.6001,-73.9662,82.38,54,96,3.00
4,Qaanaaq,GL,2022-09-03 18:11:22,77.4840,-69.3632,38.53,98,100,10.00


In [24]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")